In [1]:
import os
import pandas as pd

In [26]:
# Directory containing the text files
directory = "./Outputs_list1"

keys = ['Trial_no','Trial_cat','Cat_comb_type','Word1','Word2','Pair_no1','Pair_no2','Gender_comb','Cat_comb','']
# Loop through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith(".txt"):  # Only process .txt files
        filepath = os.path.join(directory, filename)
        try:
            # Read the text file into a DataFrame
            df = pd.read_csv(filepath, sep='\t', encoding='windows-1252')

            df = df.fillna("")
            
            # Generate the concatenated column
            df['Concatenated_Row'] = df.apply(
                lambda row: ', '.join([
                    f'{keys[i]}: "{int(value) if isinstance(value, float) and value.is_integer() else value}"'
                    for i, value in enumerate(row)
                ]), axis=1
            )
            
            # Display the updated DataFrame
            print(f"Data from {filename}:")
            print(df[['Concatenated_Row']].head())  # Show the concatenated column
        except Exception as e:
            print(f"Failed to process {filename}: {e}")


Data from Output_list1_1.txt:
                                    Concatenated_Row
0  Trial_no: "X2", Trial_cat: "Pra", Cat_comb_typ...
1  Trial_no: "X3", Trial_cat: "Pra", Cat_comb_typ...
2  Trial_no: "X4", Trial_cat: "Dis", Cat_comb_typ...
3  Trial_no: "X5", Trial_cat: "Pra", Cat_comb_typ...
4  Trial_no: "48", Trial_cat: "Exp", Cat_comb_typ...
Data from Output_list1_10.txt:
                                    Concatenated_Row
0  Trial_no: "X2", Trial_cat: "Pra", Cat_comb_typ...
1  Trial_no: "X3", Trial_cat: "Pra", Cat_comb_typ...
2  Trial_no: "X4", Trial_cat: "Dis", Cat_comb_typ...
3  Trial_no: "X5", Trial_cat: "Pra", Cat_comb_typ...
4  Trial_no: "17", Trial_cat: "Exp", Cat_comb_typ...
Data from Output_list1_2.txt:
                                    Concatenated_Row
0  Trial_no: "X2", Trial_cat: "Pra", Cat_comb_typ...
1  Trial_no: "X3", Trial_cat: "Pra", Cat_comb_typ...
2  Trial_no: "X4", Trial_cat: "Dis", Cat_comb_typ...
3  Trial_no: "X5", Trial_cat: "Pra", Cat_comb_typ...
4  Trial

In [27]:
for i in df.Concatenated_Row:
    print("{ "+i+" }")

{ Trial_no: "X2", Trial_cat: "Pra", Cat_comb_type: "between", Word1: "Flamingo", Word2: "Schnur", Pair_no1: "", Pair_no2: "", Gender_comb: "", Cat_comb: "", : "" }
{ Trial_no: "X3", Trial_cat: "Pra", Cat_comb_type: "between", Word1: "Ingwer", Word2: "Giraffe", Pair_no1: "", Pair_no2: "", Gender_comb: "", Cat_comb: "", : "" }
{ Trial_no: "X4", Trial_cat: "Dis", Cat_comb_type: "within", Word1: "Sessel", Word2: "Regal", Pair_no1: "", Pair_no2: "", Gender_comb: "", Cat_comb: "", : "" }
{ Trial_no: "X5", Trial_cat: "Pra", Cat_comb_type: "between", Word1: "Pflaume", Word2: "Trichter", Pair_no1: "", Pair_no2: "", Gender_comb: "", Cat_comb: "", : "" }
{ Trial_no: "10", Trial_cat: "Exp", Cat_comb_type: "between", Word1: "Apfel", Word2: "Besen", Pair_no1: "22221", Pair_no2: "42222", Gender_comb: "22", Cat_comb: "24", : "1" }
{ Trial_no: "31", Trial_cat: "Exp", Cat_comb_type: "between", Word1: "Schere", Word2: "Ente", Pair_no1: "43121", Pair_no2: "11112", Gender_comb: "11", Cat_comb: "41", : "1" 